In [ ]:
#@title Copyright 2021 The Earth Engine Community Authors { display-mode: "form" }
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image Overview



As mentioned in the [Get Started](https://developers.google.com/earth-engine/guides/getstarted#earth-engine-data-structures) doc, raster data are represented as `Image` objects in Earth Engine. Images are composed of one or more bands and each band has its own name, data type, scale, mask and projection. Each image has metadata stored as a set of properties.

## Setup

### Earth Engine setup

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

### Folium setup (for interactive map display)

In [ ]:
import folium


def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

##  `ee.Image` constructor 

Images can be loaded by pasting an Earth Engine asset ID into the `ee.Image` constructor. You can find image IDs in the [data catalog](https://developers.google.com/earth-engine/datasets). For example, to load [JAXA's ALOS DSM](https://developers.google.com/earth-engine/datasets/catalog/JAXA_ALOS_AW3D30_V3_2):

In [ ]:
loaded_image = ee.Image('JAXA/ALOS/AW3D30/V2_2')

You can also use a personal asset ID as shown in [this doc](https://developers.google.com/earth-engine/guides/asset_manager#importing-assets-to-your-script).

##  Get an `ee.Image` from an `ee.ImageCollection`

The standard way to get an image out of a collection is to filter the collection, with filters in order of decreasing specificity. For example, to get an image out of the [Sentinel-2 surface reflectance collection](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR):

In [ ]:
first = (ee.ImageCollection('COPERNICUS/S2_SR')
         .filterBounds(ee.Geometry.Point(-70.48, 43.3631))
         .filterDate('2019-01-01', '2019-12-31')
         .sort('CLOUDY_PIXEL_PERCENTAGE')
         .first())

# Define a map centered on southern Maine.
map_s2 = folium.Map(location=[43.7516, -70.8155], zoom_start=11)

# Add the image layer to the map and display it.
map_s2.add_ee_layer(
    first, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2000}, 'first')
display(map_s2)

Note that the sort is _after_ the filters. Avoid sorting the entire collection.

##  Images from Cloud GeoTIFFs 

You can use `ee.Image.loadGeoTIFF()` to load images from [Cloud Optimized GeoTIFFs](https://github.com/cogeotiff/cog-spec/blob/master/spec.md) in [Google Cloud Storage](https://cloud.google.com/storage). For example, the [public Landsat dataset](https://console.cloud.google.com/marketplace/details/usgs-public-data/landast) hosted in Google Cloud contains [this GeoTIFF](https://console.cloud.google.com/storage/browser/_details/gcp-public-data-landsat/LC08/01/001/002/LC08_L1GT_001002_20160817_20170322_01_T2/LC08_L1GT_001002_20160817_20170322_01_T2_B5.TIF), corresponding to band 5 from a Landsat 8 scene. You can load this image from Cloud Storage using `ee.Image.loadGeoTIFF()`:

In [ ]:
uri = ('gs://gcp-public-data-landsat/LC08/01/001/002/'
       'LC08_L1GT_001002_20160817_20170322_01_T2/'
       'LC08_L1GT_001002_20160817_20170322_01_T2_B5.TIF')
cloud_image = ee.Image.loadGeoTIFF(uri)
print(cloud_image.getInfo())

Note that if you want to reload a Cloud Optimized GeoTIFF that you [export from Earth Engine to Cloud Storage](https://developers.google.com/earth-engine/guides/exporting#to-cloud-storage), when you do the export, set `cloudOptimized` to **true** as described [here](https://developers.google.com/earth-engine/guides/exporting#configuration-parameters).

##  Constant images 

In addition to loading images by ID, you can also create images from constants, lists or other suitable Earth Engine objects. The following illustrates methods for creating images, getting band subsets, and manipulating bands:

In [ ]:
from pprint import pprint

print('Create a constant image:')
image_1 = ee.Image(1)
pprint(image_1.getInfo())

print('\nConcatenate two images into one multi-band image:')
image_2 = ee.Image(2)
image_3 = ee.Image.cat([image_1, image_2])
pprint(image_3.getInfo())

print('\nCreate a multi-band image from a list of constants:')
multiband = ee.Image([1, 2, 3])
pprint(multiband.getInfo())

print('\nSelect and (optionally) rename bands:')
renamed = multiband.select(
    ['constant', 'constant_1', 'constant_2'],  # old names
    ['band1', 'band2', 'band3'])               # new names
pprint(renamed.getInfo())

print('\nAdd bands to an image:')
image_4 = image_3.addBands(ee.Image(42))
pprint(image_4.getInfo())